In [13]:
import flowermd
import hoomd
import numpy as np
from flowermd.library.forcefields import BeadSpring
from flowermd.base import Pack, Simulation
from flowermd.library import LJChain
# importing tools

In [14]:
cpu = hoomd.device.CPU() # defining device for simulation

In [15]:
T = 1.5 # temperature for simulation
kg_chain = LJChain(lengths=10,num_mols=1) # creating a single polymer chain of length 10
ff = BeadSpring(r_cut=2.5,
        beads={"A": dict(epsilon=.1, sigma=1.0)},
        bonds={"A-A": dict(r0=1.0, k=5000.0)},
        angles={"A-A-A": dict(t0=np.pi, k=1.0)},
        dihedrals={"A-A-A-A": dict(phi0=0.0, k=0.1, d=-1, n=1)}) # forcefield with r0 = 1.0, as it is the initialized bond length. higher k 
        # will increase the stiffness of the chain
system = Pack(molecules = kg_chain, density=0.001) # creating a system with the polymer chain, low density is helpful for overlaps for multiple chains
sim = Simulation(
        initial_state=system.hoomd_snapshot,
        forcefield=ff.hoomd_forces,
        gsd_write_freq=int(1e5),
        device = cpu, gsd_file_name="single_chain.gsd") # passing in initial system snapshot, forcefield, writing frequency, device, and output file name
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=T) # initializing velocity of chain
sim.run_NVT(n_steps=1e6, kT=T, tau_kt=1) # running simulation 
sim.flush_writers() # logging data

Initializing simulation state from a gsd.hoomd.Frame.
Step 50500 of 1000000; TPS: 84016.42; ETA: 0.2 minutes
Step 101000 of 1000000; TPS: 118916.14; ETA: 0.1 minutes
Step 151500 of 1000000; TPS: 137423.94; ETA: 0.1 minutes
Step 202000 of 1000000; TPS: 149462.45; ETA: 0.1 minutes
Step 252500 of 1000000; TPS: 157725.46; ETA: 0.1 minutes
Step 303000 of 1000000; TPS: 163431.74; ETA: 0.1 minutes
Step 353500 of 1000000; TPS: 167577.56; ETA: 0.1 minutes
Step 404000 of 1000000; TPS: 170861.51; ETA: 0.1 minutes
Step 454500 of 1000000; TPS: 173647.0; ETA: 0.1 minutes
Step 505000 of 1000000; TPS: 174995.47; ETA: 0.0 minutes
Step 555500 of 1000000; TPS: 169138.02; ETA: 0.0 minutes
Step 606000 of 1000000; TPS: 164488.15; ETA: 0.0 minutes
Step 656500 of 1000000; TPS: 160888.41; ETA: 0.0 minutes
Step 707000 of 1000000; TPS: 155356.24; ETA: 0.0 minutes
Step 757500 of 1000000; TPS: 157272.69; ETA: 0.0 minutes
Step 808000 of 1000000; TPS: 155362.36; ETA: 0.0 minutes
Step 858500 of 1000000; TPS: 151885.6

In [16]:
system.visualize() # visualizing chain

/home/eridanirojas/miniconda3/envs/flowermd/lib/python3.12/site-packages/mbuild/conversion.py:1994: UserWarning: No element attribute associated with '<A pos=([ 0.2     0.5238 10.5721]), 1 bonds, id: 140681064721120>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [17]:
positions = system.hoomd_snapshot.particles.position
d = np.linalg.norm(positions[0] - positions[1])
print(f"Distance between particle 0 and 1: {d:.5f} nm")

Distance between particle 0 and 1: 1.00000 nm
